In [1]:
import os
import zipfile
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
zip_path = "/content/Bill_dataset.zip"   # your zip file path
extract_path = "dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully")

Dataset extracted successfully


In [3]:
base_dir = "data"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

classes = ["1", "5", "10", "20"]

# Create folders
for folder in [train_dir, test_dir]:
    for cls in classes:
        os.makedirs(os.path.join(folder, cls), exist_ok=True)

print("Train and Test folders created")

Train and Test folders created


In [4]:
source_dir = "dataset/Bill_dataset"

split_ratio = 0.2   # 20% test

for cls in classes:

    class_path = os.path.join(source_dir, cls)

    images = [img for img in os.listdir(class_path) if img.endswith(".tif")]

    random.shuffle(images)

    test_size = int(len(images) * split_ratio)

    test_images = images[:test_size]
    train_images = images[test_size:]

    # Move train images
    for img in train_images:
        src = os.path.join(class_path, img)
        dst = os.path.join(train_dir, cls, img)
        shutil.copy(src, dst)

    # Move test images
    for img in test_images:
        src = os.path.join(class_path, img)
        dst = os.path.join(test_dir, cls, img)
        shutil.copy(src, dst)

print("Dataset split into train and test successfully")

Dataset split into train and test successfully


In [5]:
img_size = (128,128)
batch_size = 16

train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 130 images belonging to 4 classes.
Found 31 images belonging to 4 classes.


In [6]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))   # 4 classes

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,156 (12.61 MB)

 Trainable params: 3,305,156 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 667ms/step - accuracy: 0.3858 - loss: 1.4922 - val_accuracy: 0.5806 - val_loss: 1.0116
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 680ms/step - accuracy: 0.5625 - loss: 1.0085 - val_accuracy: 0.7742 - val_loss: 0.7462
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 535ms/step - accuracy: 0.8114 - loss: 0.6271 - val_accuracy: 0.9355 - val_loss: 0.5629
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 738ms/step - accuracy: 0.8981 - loss: 0.4642 - val_accuracy: 0.9355 - val_loss: 0.3371
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 530ms/step - accuracy: 0.9603 - loss: 0.2718 - val_accuracy: 0.9355 - val_loss: 0.1702
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 540ms/step - accuracy: 0.9761 - loss: 0.1210 - val_accuracy: 1.0000 - val_loss: 0.0252
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 609ms/step - accuracy: 1.0000 - loss: 0.0221 - val_accuracy: 1.0000 - val_loss: 0.0141
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 531ms/step - accuracy: 1.0000 - loss: 0.0046 - val_accuracy: 1.0000 - val_loss:

In [9]:
test_loss, test_accuracy = model.evaluate(test_generator)

print("Test Accuracy =", test_accuracy*100, "%")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 1.0000 - loss: 1.5292e-04
Test Accuracy = 100.0 %


In [11]:
from tensorflow.keras.preprocessing import image

# Get a list of image files for class '10' from the test generator
class_10_files = [f for f in test_generator.filenames if f.startswith('10/')]

if class_10_files:
    # Pick the first image file for class '10'
    sample_image_name = class_10_files[0]
    img_path = os.path.join(test_dir, sample_image_name)
else:
    # Fallback to the original path if no images found, though this should not happen
    # if the dataset split was successful.
    img_path = "data/test/10/sample.tif"
    print("Warning: No images found for class '10' in the test set. Using a placeholder path.")

img = image.load_img(img_path, target_size=(128,128))
img_array = image.img_to_array(img)
img_array = img_array/255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)

class_names = list(train_generator.class_indices.keys())

predicted_class = class_names[np.argmax(prediction)]

print("Predicted Dollar Bill =", predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
Predicted Dollar Bill = 10
